In [1]:
from datasets import load_dataset
from transformer_lens import HookedTransformer
from sae_lens import SAE
import torch

from transformer_lens.utils import tokenize_and_concatenate
from huggingface_hub import login

/n/home04/cfang/.conda/envs/sae/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from configs import huggingface_token

In [2]:
login(token=huggingface_token)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
release = "gemma-scope-2b-pt-res-canonical"
sae_id = "layer_7/width_16k/canonical"
sae = SAE.from_pretrained(release, sae_id)[0]

In [5]:
!nvidia-smi

Sat Feb  8 14:47:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:A6:00.0 Off |                  Off |
| N/A   27C    P0            113W /  550W |   34599MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [6]:
model = HookedTransformer.from_pretrained("gemma-2-2b", device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  6.31it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loaded pretrained model gemma-2-2b into HookedTransformer


In [7]:
!nvidia-smi

Sat Feb  8 14:47:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:A6:00.0 Off |                  Off |
| N/A   27C    P0            114W /  550W |   49661MiB /  81559MiB |      5%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   1  NVIDIA H100 80GB HBM3          On  |   00000000:C6:00.0 Off |                  Off |
| N/A   24C    P0             68W /  550W |       5MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A    673867      C   ...04/cfang/.conda/envs/sae/bin/python      3

# Load feature interpretations

# Loading and running on Pile

In [8]:
dataset = load_dataset(
    path="NeelNanda/pile-10k",
    split="train",
    streaming=False,
)

Generating train split: 100%|██████████| 10000/10000 [00:01<00:00, 6707.25 examples/s] 


In [13]:
token_dataset = tokenize_and_concatenate(
    dataset=dataset,  # type: ignore
    tokenizer=model.tokenizer,  # type: ignore
    streaming=True,
    max_length=sae.cfg.context_size,
    add_bos_token=sae.cfg.prepend_bos,
)

Map: 100%|██████████| 10000/10000 [00:05<00:00, 1714.25 examples/s]


In [1]:
batch_tokens = token_dataset[:32]["tokens"]
_, cache = model.run_with_cache(
    batch_tokens,
    prepend_bos=True,
    names_filter=lambda name: name.startswith("blocks.7.")
)


NameError: name 'token_dataset' is not defined

In [ ]:
cache.keys()